<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">tf.data بازنویسی نوت بوک 7 با </div></center></h1>
                         rasoul asadiyan  

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os, shutil
import pathlib

In [3]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

68616192/68606236 [==============================] - 1s 0us/step


In [119]:
PATH

'/root/.keras/datasets/cats_and_dogs_filtered'

In [180]:
full_data=tf.data.Dataset.list_files(str('/root/.keras/datasets/cats_and_dogs_filtered/*/*/*'))

In [136]:
for i in full_data.take(5):
  print(i)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.990.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.475.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.318.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.321.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.37.jpg', shape=(), dtype=string)


In [130]:
full_train=tf.data.Dataset.list_files(str('/root/.keras/datasets/cats_and_dogs_filtered/train/*/*'))

In [131]:
for i in full_train.take(5):
  print(i)

tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.61.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.412.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.964.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.833.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.628.jpg', shape=(), dtype=string)


In [132]:
full_validation=tf.data.Dataset.list_files(str('/root/.keras/datasets/cats_and_dogs_filtered/validation/*/*'))

In [138]:
dataset_size=len(list(full_data))# train=2000 / validation=1000
train_size=len(list(full_train))
validation_size=len(list(full_validation))
print("full_dataset_size = {}".format(dataset_size))
print("full_train_size = {}".format(train_size))
print("full_validation_size = {}".format(validation_size))

full_dataset_size = 3000
full_train_size = 2000
full_validation_size = 1000


In [140]:
class_names=np.array(['cats','dogs'])

In [175]:
class_names

array(['cats', 'dogs'], dtype='<U4')

In [181]:
def augment_data(img):
  img=tf.image.resize_with_crop_or_pad(img,180,180)
  img=tf.image.random_crop(img,size=[150,150,3])
  img=tf.image.random_brightness(img,max_delta=0.5)
  
  return img

In [273]:
def get_label(file_path):
  parts=tf.strings.split(file_path,os.path.sep)
  if parts[-2]=='cats':
    return 0
  else:
    return 1  

In [274]:
get_label('root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.412.jpg')

1

In [275]:
def load_img(imag_path):
  img=tf.io.read_file(imag_path)
  img=tf.image.decode_image(img,3,expand_animations=False)
  img=tf.cast(img,tf.float32)
  
  return img

In [276]:
def normalize(img):
  img=(img/127.5)-1
  return img

In [277]:
def resize_img(imag,height,width):
  img=tf.image.resize(imag,(height,width),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return img

In [278]:
def load_img_label(img_path):
  imag=load_img(img_path)
  label=get_label(img_path)
  return imag,label

In [279]:
def load_imag_train(img_path):
  imag,label=load_img_label(img_path)
  imag=augment_data(imag)
  imag=normalize(imag)

  return imag,label

In [280]:
def load_img_validation(img_path):
  imag,label=load_img_label(img_path)
  imag=resize_img(imag,150,150)
  imag=normalize(imag)

  return imag,label

In [281]:
batch_size=32
shuffle_buffer_size=500

In [282]:
train_dataset=full_train.map(load_imag_train)
train_dataset=train_dataset.shuffle(shuffle_buffer_size)
train_dataset=train_dataset.batch(batch_size)

In [283]:
validation_dataset=full_validation.map(load_img_validation)
validation_dataset=validation_dataset.batch(batch_size)

In [284]:
base_model=tf.keras.applications.VGG16(weights='imagenet',include_top=False,input_shape=(150,150,3))
base_model.trainable=False

In [285]:
flatten_layer = tf.keras.layers.GlobalAveragePooling2D()
# conv_layer=tf.keras.layers.Conv2D(32,(3,3),activation='relu')
dense_layer = tf.keras.layers.Dense(100, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
flatten = tf.keras.layers.Flatten()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [286]:
model = tf.keras.Sequential([
  base_model,
  flatten_layer, dense_layer, dropout_layer,
  flatten,
  prediction_layer
])

In [287]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d_17  (None, 512)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_28 (Dense)            (None, 100)               51300     
                                                                 
 dropout_16 (Dropout)        (None, 100)               0         
                                                                 
 flatten_4 (Flatten)         (None, 100)               0         
                                                                 
 dense_29 (Dense)            (None, 1)                 101       
                                                      

In [288]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [289]:
for data_batch,label_batch in validation_dataset:
  print(data_batch.shape)
  print(label_batch.shape)
  
  break

(32, 150, 150, 3)
(32,)


In [290]:
h = model.fit(train_dataset,
                    epochs=10,
                    validation_data=validation_dataset)

Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.5922 - accuracy: 0.7400

KeyboardInterrupt: ignored